In [35]:
import psycopg2
import json
from datetime import datetime as dt
import pandas as pd
import re 
from textblob import TextBlob

In [ ]:
p="dataisok"

In [17]:
conn = psycopg2.connect(user = "sam",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

In [18]:
cursor = conn.cursor()

In [138]:
tweet_list = []
followers = []
cursor.execute("select * from tweets")
tweets = cursor.fetchall()
if not tweets:
    print("empty")
for row in tweets:
    for col in row:
        if type(col) is dict:
            tweet_list.append(col['text'])
            followers.append(col['user']['followers_count'])

In [139]:
remove_list = [r'@[A-Za-z0-9]+','https?://[A-Za-z0-9./]+','\n','RT :',r'_[A-Za-z0-9]+','  ']

In [140]:
def clean_tweets(remove_list,tweet_list):
    for item in remove_list:
        tweet_list = [re.sub(item,'',tweet) for tweet in tweet_list]
    return tweet_list

In [141]:
cleaned = clean_tweets(remove_list,tweet)
cleaned

['There are so many hilarious comments in You Tubechat on @ show tonight.',
 ' Democrat frontrunner Michael Bloomberg says he wants to make the whole country "behave" while dressed as Mary Poppins.Wel…',
 'The latest COACH!#election2020 #misleadingtweets',
 'THERE IS NO ONE TO VOTE FOR! Centrists demand Valium to get them through #Election2020',
 'RT : Calling all voters!! Are you ready for November?👉🏿RT to spread the word. 🗳️ #Election2020 http…',
 ' Team Warren! #PresidentWarren #DemocraticPrimaries #Election2020 ',
 'RT :#Election2020 Enjoy…',
 ' As a man and feminist I am flabbergasted at this Tweet from you. This says so much about your own bi… ',
 " It's $3 Thursday! I'm asking for everyone to help that is able to donate $3 to my campaign for Congress. Join us and help put p…",
 '2020 Presidential Election News Update🗞:2020 Democratic contenders plead for cash after draining their accounts… ',
 ' #Bloomberg is buying the Dem nomination *because* many resisters won’t support him👀‼

In [154]:
sentiment_objects = []
for each in cleaned:
    sentiment_objects.append([(each),TextBlob(each).polarity])

    
# sentiment_objects.polarity, sentiment_objects
print(sentiment_objects[0])
    


['There are so many hilarious comments in You Tubechat on @ show tonight.', 0.5]


In [11]:
i

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [14]:
my_dict = {"Key":1}

In [16]:
"Key" in my_dict.keys()

True

In [17]:
emolex_df['emotion'].iloc[4]

'surprise'

In [73]:
import nltk
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples
from nltk.stem.wordnet import WordNetLemmatizer
import re, string
from nltk import FreqDist
import random
import psycopg2
from textblob import TextBlob 
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('twitter_samples')

stop_words = stopwords.words('english')


text = twitter_samples.strings('tweets.20150430-223406.json')
twitter_samples

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]
['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [74]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [ ]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))


def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)


freq_dist_pos = FreqDist(all_pos_words)
# print(freq_dist_pos.most_common(10))

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

p="datasucks"
conn = psycopg2.connect(user = "alexis",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

cursor = conn.cursor()
tweet_list = []
followers = []
cursor.execute("select * from tweets")
tweets = cursor.fetchall()
if not tweets:
    print("empty")
for row in tweets:
    for col in row:
        if type(col) is dict:
            tweet_list.append(col['text'])
            followers.append(col['user']['followers_count'])

remove_list = [r'@[A-Za-z0-9]+','https?://[A-Za-z0-9./]+','\n','RT :',r'_[A-Za-z0-9]+','  ']

In [39]:
def clean_tweets(remove_list,tweet_list):
    for item in remove_list:
        tweet_list = [re.sub(item,'',tweet) for tweet in tweet_list]
    return tweet_list

cleaned = clean_tweets(remove_list,tweet_list)

#NLTK Sentiment addition to tweets, tweets cleaned but untokenized?

In [40]:
tweet = []
sentiment = []
for each in cleaned:
    custom_tweet = each
    tweet.append(custom_tweet)
    custom_tokens = remove_noise(word_tokenize(custom_tweet))
#     sentiment.append(classifier.classify(dict([token, True] for token in custom_tokens)))
                                         
    # print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

# clean_sentiment = pd.DataFrame(tweet, sentiment).reset_index()
# clean_sentiment = clean_sentiment.rename(columns = {"index":"NLTK sentiment",0:"tweet"})

In [41]:
def tokenize_this_list(text):
    token_list = []
    token_count = []
    for each in text:
        tokens = word_tokenize(each)
        count = len(tokens)
        token_list.append(tokens)
        token_count.append(count)
    return token_count, token_list

token_count, token_list = tokenize_this_list(tweet)

print('Length of Token_Count =' + str(len(token_count)))
print('Length of Token_List =' + str(len(token_list)))

Length of Token_Count =2316
Length of Token_List =2316


In [42]:
# NRC
filepath = "../NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t')
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()


In [66]:
def full_list(token_list, df):
    # Set list to hold dictionary and return
    emo_list = []
    # Loop through tweets in list
    for tweet in token_list:
        # Set dictionary to hold emotions as keys and their counts as values
        lexi_dict = {}
        # Loop through words in tweet
        for word in tweet:
            # Check if word is in df -> emotion lexicon dataframe (emolex_df) in our example
            if word in df.word.to_list():
                # List to hold index where association column is 1
                assoc = []
                # Get indices where word from tweet is in df
                i = df.index[df['word'] == word]
                for index in i:
                    # Check if association is 1 (not 0) and append to list
                    if df['association'].iloc[index] == 1:
                        assoc.append(index)
                    # Where the word from tweet has association = 1, add to dict if not there, or update count if there
                    for j in assoc:
                        emo = df['emotion'].iloc[j]
                        if (emo in lexi_dict.keys()):
                            lexi_dict[emo] += 1
                        else:
                            lexi_dict[emo] = 1
        # Append to list
        emo_list.append(lexi_dict)
    # return list
    return emo_list

emotion_list = full_list(token_list,emolex_df)
               
print(emotion_list[:10])






[{'joy': 6, 'positive': 4, 'surprise': 2, 'trust': 1}, {}, {}, {'anger': 10, 'negative': 5}, {'anticipation': 9, 'positive': 4, 'trust': 1}, {}, {}, {}, {}, {'anger': 10, 'anticipation': 9, 'fear': 7, 'joy': 6, 'positive': 4, 'trust': 1}]


In [67]:
token_list[:10]

[['There',
  'are',
  'so',
  'many',
  'hilarious',
  'comments',
  'in',
  'You',
  'Tubechat',
  'on',
  '@',
  'show',
  'tonight',
  '.'],
 ['Democrat',
  'frontrunner',
  'Michael',
  'Bloomberg',
  'says',
  'he',
  'wants',
  'to',
  'make',
  'the',
  'whole',
  'country',
  '``',
  'behave',
  "''",
  'while',
  'dressed',
  'as',
  'Mary',
  'Poppins.Wel…'],
 ['The', 'latest', 'COACH', '!', '#', 'election2020', '#', 'misleadingtweets'],
 ['THERE',
  'IS',
  'NO',
  'ONE',
  'TO',
  'VOTE',
  'FOR',
  '!',
  'Centrists',
  'demand',
  'Valium',
  'to',
  'get',
  'them',
  'through',
  '#',
  'Election2020'],
 ['RT',
  ':',
  'Calling',
  'all',
  'voters',
  '!',
  '!',
  'Are',
  'you',
  'ready',
  'for',
  'November',
  '?',
  '👉🏿RT',
  'to',
  'spread',
  'the',
  'word',
  '.',
  '🗳️',
  '#',
  'Election2020',
  'http…'],
 ['Team',
  'Warren',
  '!',
  '#',
  'PresidentWarren',
  '#',
  'DemocraticPrimaries',
  '#',
  'Election2020'],
 ['RT', ':', '#', 'Election2020', '

In [68]:
score_df = pd.DataFrame({"Tokens":token_list,"Emotions":emotion_list})

In [69]:
score_df.head()

,Tokens,Emotions
0,"[There, are, so, many, hilarious, comments, in...","{'joy': 6, 'positive': 4, 'surprise': 2, 'trus..."
1,"[Democrat, frontrunner, Michael, Bloomberg, sa...",{}
2,"[The, latest, COACH, !, #, election2020, #, mi...",{}
3,"[THERE, IS, NO, ONE, TO, VOTE, FOR, !, Centris...","{'anger': 10, 'negative': 5}"
4,"[RT, :, Calling, all, voters, !, !, Are, you, ...","{'anticipation': 9, 'positive': 4, 'trust': 1}"


In [72]:
emolex_df.iloc[59772]

word           hilarious
emotion         positive
association            1
Name: 59772, dtype: object

In [63]:
lexi_dict = {}
for word in token_list[0]:
    print(word)
    if word in emolex_df.word.to_list():
        assoc = []
        index = emolex_df.index[emolex_df['word'] == word]
        for i in index:
            print(i)
            if emolex_df['association'].iloc[i] == 1:
                assoc.append(i)
                print(assoc)
            for j in assoc:
                emo = emolex_df['emotion'].iloc[j]
                print(emo)
                print(word)
                print(emolex_df.iloc[j])
                if (emo in lexi_dict.keys()):
                    lexi_dict[emo] += 1
                else:
                    lexi_dict[emo] = 1

There
are
so
many
hilarious
59766
59767
59768
59769
59770
[59770]
joy
hilarious
word           hilarious
emotion              joy
association            1
Name: 59770, dtype: object
59771
joy
hilarious
word           hilarious
emotion              joy
association            1
Name: 59770, dtype: object
59772
[59770, 59772]
joy
hilarious
word           hilarious
emotion              joy
association            1
Name: 59770, dtype: object
positive
hilarious
word           hilarious
emotion         positive
association            1
Name: 59772, dtype: object
59773
joy
hilarious
word           hilarious
emotion              joy
association            1
Name: 59770, dtype: object
positive
hilarious
word           hilarious
emotion         positive
association            1
Name: 59772, dtype: object
59774
[59770, 59772, 59774]
joy
hilarious
word           hilarious
emotion              joy
association            1
Name: 59770, dtype: object
positive
hilarious
word           hilarious
emotion

In [64]:
lexi_dict

{'joy': 6, 'positive': 4, 'surprise': 2, 'trust': 1}

In [70]:
6/13

0.46153846153846156